# DATA PARALLELISM

It’s natural to execute your forward, backward propagations on multiple GPUs. 

However, Pytorch will only use one GPU by default. 

You can easily run your operations on multiple GPUs by making your model run parallelly using DataParallel:

model = nn.DataParallel(model)

In [5]:
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader

# Parameters and DataLoaders
input_size = 5
output_size = 2

batch_size = 30
data_size = 100

In [6]:
# device
device = torch.device("mps")

### Make dummy dataset

In [7]:
class RandomDataset(Dataset):
    def __init__(self, size, length):
        self.len = length 
        self.data = torch.randn(length, size)
        
    def __getitem__(self, index):
        return self.data[index]

    def __len__(self):
        return self.len

In [8]:
rand_loader = DataLoader(dataset=RandomDataset(input_size, data_size),
                         batch_size=batch_size, shuffle=True)

In [9]:
# simple model

class Model(nn.Module):
    # Our model

    def __init__(self, input_size, output_size):
        super(Model, self).__init__()
        self.fc = nn.Linear(input_size, output_size)

    def forward(self, input):
        output = self.fc(input)
        print("\tIn Model: input size", input.size(),
              "output size", output.size())

        return output

In [10]:
# Create Model and DataParallel
model = Model(input_size, output_size)
model = nn.DataParallel(model)
model.to(device)

DataParallel(
  (module): Model(
    (fc): Linear(in_features=5, out_features=2, bias=True)
  )
)

In [11]:
# Run the model
for data in rand_loader:
    input = data.to(device)
    output = model(input)
    print("Outside: input size", input.size(),
          "output_size", output.size())


	In Model: input size torch.Size([30, 5]) output size torch.Size([30, 2])
Outside: input size torch.Size([30, 5]) output_size torch.Size([30, 2])
	In Model: input size torch.Size([30, 5]) output size torch.Size([30, 2])
Outside: input size torch.Size([30, 5]) output_size torch.Size([30, 2])
	In Model: input size torch.Size([30, 5]) output size torch.Size([30, 2])
Outside: input size torch.Size([30, 5]) output_size torch.Size([30, 2])
	In Model: input size torch.Size([10, 5]) output size torch.Size([10, 2])
Outside: input size torch.Size([10, 5]) output_size torch.Size([10, 2])
